# Utilização de LLMs

In [1]:
import logging
import warnings

# Suprimir avisos específicos de FutureWarning e UserWarning
warnings.filterwarnings("ignore", category=FutureWarning, message=".*TRANSFORMERS_CACHE.*")
warnings.filterwarnings("ignore", message=".*resume_download.*deprecated.*", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*use_cache=True.*", category=UserWarning)
warnings.filterwarnings("ignore", message=".*use_reentrant parameter should be passed explicitly.*", category=UserWarning)
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly.")


# Configurar o nível de log para a biblioteca transformers
logging.getLogger("transformers.trainer").setLevel(logging.WARNING)
logging.getLogger("transformers.trainer_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.training_args").setLevel(logging.WARNING)




In [2]:
import torch
import bitsandbytes
import peft
import accelerate
import datasets
import trl
import warnings
import transformers

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print("transformers version:", transformers.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

torch version: 2.3.1
bitsandbytes version: 0.43.1
peft version: 0.11.1
accelerate version: 0.31.0
datasets version: 2.19.2
trl version: 0.9.6
transformers version: 4.44.0
Device name: 'NVIDIA GeForce RTX 2060 SUPER'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060 SUPER', major=7, minor=5, total_memory=7957MB, multi_processor_count=34)'
Suporta bfloat16.


In [3]:
import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset


from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback,
    set_seed,
    pipeline,
    TrainerCallback,
    TrainerControl,
    TrainerState,
)

In [4]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [5]:
# if torch.cuda.is_bf16_supported():
#     compute_dtype = torch.bfloat16
#     # attn_implementation = 'flash_attention_2'
#     attn_implementation = 'eager'
# else:
compute_dtype = torch.float16
attn_implementation = 'eager'

print(attn_implementation)
print(compute_dtype)

eager
torch.float16


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token=os.environ["HUGGINGFACE_TOKEN"])

model_id = "emdemor/question-generator"


# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)

# adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
model = prepare_model_for_kbit_training(model)

def set_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.padding_side = 'right'
    return tokenizer
tokenizer = set_tokenizer(model_id)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

In [7]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

context = "Em nota a família informou que o desejo de Silvio Santos em relação a sua morte era de que quando partisse, fosse levado diretamente ao cemitério particular onde fosse realizada uma cerimônia judaica. Silvio Santos também havia desejado que sua morte não fosse explorada, pois gostava de ser celebrado em vida. Gostaria de ser lembrado com a alegria que viveu e que seu desejo fosse respeitado."


messages = [
    {
        "content": "Você é um assistente especializado em interpretação de texto",
        "role": "system"
    },
    {
        "content": f"Gere uma pergunta para o seguinte contexto:\n```\n{context}\n```\nPergunta:",
        "role": "user"
    },
]

with torch.cuda.amp.autocast():
    # model.generate(**tokenizer("test", return_tensors="pt").to("cuda"))
    result = llm.generate(messages)

result

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


CPU times: user 1.63 s, sys: 107 ms, total: 1.74 s
Wall time: 1.74 s


'Em que sentimento Silvio Santos gostava em relação à sua morte?'

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [15]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

context = """
Dois jovens de 19 e 23 anos foram presos na noite desta sexta-feira,16, após disparos de arma de fogo próximo a um bar em Poços de Caldas.

Segundo a Polícia Militar, durante uma operação na Avenida Marechal Castelo Branco, no bairro Jardim São Paulo, os policiais ouviram dois disparos de arma de fogo.

Clique aqui e participe do grupo de notícias da Onda Poços no WhatsApp

Em seguida foi recebida uma denúncia, via 190, relatando que o jovem de 19 anos trajando camiseta do flamengo havia efetuado dois disparos de arma de fogo próximo a um bar e que ele havia repassado a arma a um segundo indivíduo que trajava blusa com capuz preta e bermuda de cor preta, que após pegar arma saiu correndo em direção à Rua Engenheiro Ubirajara Machado de Moraes.

 

Ainda segundo a PM, o jovem de 19 anos já conhecido no meio policial por posse ilegal de arma e tráfico de drogas, foi localizado e abordado.

 

Uma outra equipe da Polícia Militar recebeu denúncias, via 190, de que o segundo suspeito que havia fugido do local com a arma de fogo, a deixou em uma lixeira na Rua Abrieiro. A arma foi apreendida.

 

Ainda segundo a PM, após capturar o primeiro suspeito, a equipe continuou o rastreamento para localizar o segundo envolvido. O jovem foi encontrado ao final da Rua Engenheiro Ubirajara Machado de Moraes com as mesmas vestes repassadas pela denúncia.

 
A arma de fogo se tratava de um revólver calibre 38 marca Taurus, que foi apreendida. Ainda segundo a PM, no local e nas adjacências não foram localizadas vítimas e não foi detectado o local exato em que o projétil atingiu.

A dupla foi presa, encaminhada à UPA e em seguida levada para a Delegacia de Polícia Civil.
"""


messages = [
    {
        "content": "Você é um assistente especializado em interpretação de texto",
        "role": "system"
    },
    {
        "content": f"Gere uma pergunta para o seguinte contexto:\n```\n{context}\n```\nPergunta:",
        "role": "user"
    },
]

with torch.cuda.amp.autocast():
    # model.generate(**tokenizer("test", return_tensors="pt").to("cuda"))
    question = llm.generate(messages)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Dado o seguinte contexto ```{context}``` responda a pergunta: `{question}`."}
  ]
)

print("question =", question)
print("response =", completion.choices[0].message.content)

question = Onde a arma foi colocada no final da rua onde o segundo suspeito foi encontrado?
response = A arma foi colocada em uma lixeira na Rua Abrieiro.
CPU times: user 1.63 s, sys: 171 ms, total: 1.8 s
Wall time: 3.05 s


In [31]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

context = """
Neste artigo, apresentamos uma derivação matemática da equação de Schrödinger partindo de apenas dois axiomas. Mostramos também que, utilizando este processo de derivação formal, é possível derivar diretamente a equação de Schrödinger em sistemas de coordenadas curvilíneas generalizadas. Esta derivação também se mostra equivalente à abordagem de integrais de trajetória de Feynman, mas vai além, permitindo-nos derivar matematicamente as regras de quantização de Bohr-Sommerfeld. O uso de um parâmetro pequeno, tanto na presente derivação, em que aparece como δr, quanto na derivação de Feynman, em que aparece como ϵ = δt, também é esclarecido em termos do Teorema do Limite Central. O artigo faz, pois, uma transposição didática do tema da quantização, permitindo que seja abordado no contexto do ensino de Mecânica Quântica. A importância epistemológica de abordagens axiomáticas para a derivação matemática e a interpretação dos símbolos da teoria também é tratada.
"""


messages = [
    {
        "content": "Você é um assistente especializado em interpretação de texto",
        "role": "system"
    },
    {
        "content": f"Gere uma pergunta para o seguinte contexto:\n```\n{context}\n```\nPergunta:",
        "role": "user"
    },
]

with torch.cuda.amp.autocast():
    # model.generate(**tokenizer("test", return_tensors="pt").to("cuda"))
    question = llm.generate(messages)
    print("question =", question)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Dado o seguinte contexto ```{context}``` responda a pergunta: `{question}`."}
  ]
)


print("response =", completion.choices[0].message.content)

question = Qual é a importância epistemológica dessa abordagem da derivação matemática da Mecânica Quântica?
response = A importância epistemológica da abordagem da derivação matemática da Mecânica Quântica, conforme descrito no artigo, reside em vários aspectos:

1. **Fundamentação Teórica**: Ao partir de axiomas para derivar a equação de Schrödinger, a abordagem reforça a ideia de que é possível construir teorias físicas de forma rigorosa e estruturada, o que é fundamental para a compreensão profunda dos conceitos subjacentes à Mecânica Quântica.

2. **Conexão com Métodos Alternativos**: A equivalência entre a derivação matemática apresentada e a abordagem de integrais de trajetória de Feynman destaca a versatilidade das interpretações e metodologias dentro da Mecânica Quântica. Isso enriquece a compreensão dos estudantes ao mostrar que diferentes abordagens podem levar aos mesmos resultados, promovendo uma visão mais ampla da teoria.

3. **Contextualização no Ensino**: A transposiçã